In [0]:
spark.conf.set(
  "fs.azure.account.key.icicicasestudy.blob.core.windows.net",
  "USUXvU+0RoKNsGMv4pqgKbo6hByFjuNmiHDPV8sGxHkHbmr0BfZR8TmcNF5Q1AYOOPHKydOz9Z8X+AStgIVDxA=="
)

In [0]:
df = spark.read \
    .option("multiLine", True) \
    .json("wasbs://icici-raw@icicicasestudy.blob.core.windows.net/insurance_claims.json")

df.printSchema()
df.show(truncate=False)

root
 |-- claim_date: string (nullable = true)
 |-- claim_id: string (nullable = true)
 |-- claimant: struct (nullable = true)
 |    |-- customer_id: string (nullable = true)
 |    |-- region: string (nullable = true)
 |-- garage: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- hospital: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- payment: struct (nullable = true)
 |    |-- payment_mode: string (nullable = true)
 |    |-- status: string (nullable = true)
 |-- policy: struct (nullable = true)
 |    |-- coverage_amount: long (nullable = true)
 |    |-- insurance_type: string (nullable = true)
 |    |-- policy_number: string (nullable = true)
 |-- treatment: struct (nullable = true)
 |    |-- claim_amount: long (nullable = true)
 |    |-- damage_type: string (nullable = true)
 |    |-- diagnosis: string (nullable = true)

+----------+--------+----------------

In [0]:
from pyspark.sql.functions import col

flattened_df = df.select(
    col("claim_id"),
    col("policy.policy_number").alias("policy_number"),
    col("policy.insurance_type").alias("insurance_type"),
    col("policy.coverage_amount").alias("coverage_amount"),
    col("claimant.customer_id").alias("customer_id"),
    col("claimant.region").alias("region"),
    col("hospital.name").alias("hospital_name"),
    col("hospital.city").alias("hospital_city"),
    col("garage.name").alias("garage_name"),
    col("garage.city").alias("garage_city"),
    col("treatment.claim_amount").alias("claim_amount"),
    col("payment.status").alias("payment_status"),
    col("payment.payment_mode").alias("payment_mode"),
    col("claim_date")
)

flattened_df.show(truncate=False)


+--------+-------------+--------------+---------------+-----------+------+---------------+-------------+-----------------------+-----------+------------+--------------+------------+----------+
|claim_id|policy_number|insurance_type|coverage_amount|customer_id|region|hospital_name  |hospital_city|garage_name            |garage_city|claim_amount|payment_status|payment_mode|claim_date|
+--------+-------------+--------------+---------------+-----------+------+---------------+-------------+-----------------------+-----------+------------+--------------+------------+----------+
|CLM1001 |POL9001      |Health        |500000         |ICICI_C101 |South |Apollo Hospital|Chennai      |NULL                   |NULL       |120000      |APPROVED      |NEFT        |2024-01-15|
|CLM1002 |POL9002      |Vehicle       |300000         |ICICI_C102 |West  |NULL           |NULL         |ICICI Authorized Garage|Mumbai     |80000       |REJECTED      |UPI         |2024-01-16|
+--------+-------------+-----------

In [0]:
approved_df = flattened_df.filter(
    col("payment_status") == "APPROVED"
)

approved_df.show()

+--------+-------------+--------------+---------------+-----------+------+---------------+-------------+-----------+-----------+------------+--------------+------------+----------+
|claim_id|policy_number|insurance_type|coverage_amount|customer_id|region|  hospital_name|hospital_city|garage_name|garage_city|claim_amount|payment_status|payment_mode|claim_date|
+--------+-------------+--------------+---------------+-----------+------+---------------+-------------+-----------+-----------+------------+--------------+------------+----------+
| CLM1001|      POL9001|        Health|         500000| ICICI_C101| South|Apollo Hospital|      Chennai|       NULL|       NULL|      120000|      APPROVED|        NEFT|2024-01-15|
+--------+-------------+--------------+---------------+-----------+------+---------------+-------------+-----------+-----------+------------+--------------+------------+----------+



In [0]:
sfOptions = {
    "sfURL": "QYQSSEQ-UT92541.snowflakecomputing.com",
    "sfDatabase": "ICICI_DB",
    "sfSchema": "PUBLIC",
    "sfWarehouse": "ICICI_WH",
    "sfRole": "SYSADMIN",
    "sfUser": "BARATH46",
    "sfPassword": "Rbarathraj@2711"
}


In [0]:
approved_df.write \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "ICICI_APPROVED_CLAIMS") \
    .mode("overwrite") \
    .save()